In [81]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [82]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')
minimum_money = pd.read_excel('연봉데이터/연도별_최저연봉.xlsx')

In [83]:
print(f'투수데이터 내 결측치가 존재하는 변수')
print(pitcher.isnull().sum()[pitcher.isnull().sum() != 0])
print()
print(f'타자데이터 내 결측치가 존재하는 변수')
print(hitter.isnull().sum()[hitter.isnull().sum() != 0])

투수데이터 내 결측치가 존재하는 변수
연봉     565
WAR     43
dtype: int64

타자데이터 내 결측치가 존재하는 변수
연봉     595
WAR     35
dtype: int64


# 1. 최저연봉을 통한 신인연봉 결측치 채우기
 - 타자: 566명 중 221명 구제가능
 - 투수: 596명 중 212명 구제가능

In [84]:
# 타자에 적용
for idx in hitter.loc[(hitter['데뷔년도'] == hitter['연도']) & hitter['연봉'].isnull()].index:
    hitter.loc[idx, '연봉'] = minimum_money.loc[minimum_money['연도'] == hitter.loc[idx, '데뷔년도'], '최저연봉'].values[0]

# 투수에 적용
for idx in pitcher.loc[(pitcher['데뷔년도'] == pitcher['연도']) & pitcher['연봉'].isnull()].index:
    pitcher.loc[idx, '연봉'] = minimum_money.loc[minimum_money['연도'] == pitcher.loc[idx, '데뷔년도'], '최저연봉'].values[0]

In [85]:
print(f'투수데이터 내 결측치가 존재하는 변수')
print(pitcher.isnull().sum()[pitcher.isnull().sum() != 0])
print()
print(f'타자데이터 내 결측치가 존재하는 변수')
print(hitter.isnull().sum()[hitter.isnull().sum() != 0])

투수데이터 내 결측치가 존재하는 변수
연봉     344
WAR     43
dtype: int64

타자데이터 내 결측치가 존재하는 변수
연봉     384
WAR     35
dtype: int64


# 2. 수기조사한 데이터로 연봉데이터 결측치 채우기

In [86]:
# 데이터 불러오기
hit_null = pd.read_csv('연봉데이터/타자_연봉결측치수기조사필요.csv', encoding='cp949')[['ID','선수명','생년월일','연도','연봉']]
pit_null = pd.read_csv('연봉데이터/투수_연봉결측치수기조사필요.csv', encoding='cp949')[['ID','선수명','생년월일','연도','연봉']]

# 결측치 전처리
pit_null.loc[pit_null['연봉'].isin(['기록없음','?']), '연봉'] = np.NaN
hit_null.loc[hit_null['연봉'].isin(['기록없음','?']), '연봉'] = np.NaN

# 결측행은 제거
hit_null = hit_null.dropna().drop_duplicates().reset_index(drop=True)
pit_null = pit_null.dropna().drop_duplicates().reset_index(drop=True)

print(f'타자데이터의 개수: {hit_null.shape[0]}')
print(f'투수데이터의 개수: {pit_null.shape[0]}')

타자데이터의 개수: 297
투수데이터의 개수: 268


In [87]:
# 타자대체
for ID, YEAR, MONEY in zip(hitter['ID'], hitter['연도'], hitter['연봉']):
    if pd.isnull(MONEY):
        new_money = hit_null.loc[(hit_null['ID'] == ID) & (hit_null['연도'] == YEAR), '연봉']
        if new_money.shape[0] == 0:
            pass
        else:
            hitter.loc[(hitter['ID'] == ID) & (hitter['연도'] == YEAR), '연봉'] = int(new_money)

# 투수대체
for ID, YEAR, MONEY in zip(pitcher['ID'], pitcher['연도'], pitcher['연봉']):
    if pd.isnull(MONEY):
        new_money = pit_null.loc[(pit_null['ID'] == ID) & (pit_null['연도'] == YEAR), '연봉']
        if new_money.shape[0] == 0:
            pass
        else:
            pitcher.loc[(pitcher['ID'] == ID) & (pitcher['연도'] == YEAR), '연봉'] = int(new_money)
        
print(f'투수데이터 내 결측치가 존재하는 변수')
print(pitcher.isnull().sum()[pitcher.isnull().sum() != 0])
print()
print(f'타자데이터 내 결측치가 존재하는 변수')
print(hitter.isnull().sum()[hitter.isnull().sum() != 0])

투수데이터 내 결측치가 존재하는 변수
연봉     76
WAR    43
dtype: int64

타자데이터 내 결측치가 존재하는 변수
연봉     87
WAR    35
dtype: int64


# 3. 수기조사한 데이터로 WAR 결측치 채우기

In [88]:
# 데이터 불러오기
hit_null = pd.read_csv('WAR결측치수기조사데이터/WAR결측치(타자)_수기조사필요.csv', encoding='cp949')[['ID','연도','WAR']]
pit_null = pd.read_csv('WAR결측치수기조사데이터/WAR결측치(투수)_수기조사필요.csv', encoding='cp949')[['ID','연도','WAR']]

# 결측행은 제거
hit_null = hit_null.dropna().drop_duplicates().reset_index(drop=True)
pit_null = pit_null.dropna().drop_duplicates().reset_index(drop=True)

print(f'타자데이터의 개수: {hit_null.shape[0]}')
print(f'투수데이터의 개수: {pit_null.shape[0]}')

타자데이터의 개수: 32
투수데이터의 개수: 43


In [89]:
# 타자대체
for ID, YEAR, WAR in zip(hitter['ID'], hitter['연도'], hitter['WAR']):
    if pd.isnull(WAR):
        new_WAR = hit_null.loc[(hit_null['ID'] == ID) & (hit_null['연도'] == YEAR), 'WAR']
        if new_WAR.shape[0] == 0:
            pass
        else:
            hitter.loc[(hitter['ID'] == ID) & (hitter['연도'] == YEAR), 'WAR'] = int(new_WAR)

# 투수대체
for ID, YEAR, WAR in zip(pitcher['ID'], pitcher['연도'], pitcher['WAR']):
    if pd.isnull(WAR):
        new_WAR = pit_null.loc[(pit_null['ID'] == ID) & (pit_null['연도'] == YEAR), 'WAR']
        if new_WAR.shape[0] == 0:
            pass
        else:
            pitcher.loc[(pitcher['ID'] == ID) & (pitcher['연도'] == YEAR), 'WAR'] = int(new_WAR)
        
print(f'투수데이터 내 결측치가 존재하는 변수')
print(pitcher.isnull().sum()[pitcher.isnull().sum() != 0])
print()
print(f'타자데이터 내 결측치가 존재하는 변수')
print(hitter.isnull().sum()[hitter.isnull().sum() != 0])

투수데이터 내 결측치가 존재하는 변수
연봉    76
dtype: int64

타자데이터 내 결측치가 존재하는 변수
연봉     87
WAR     3
dtype: int64


In [91]:
hitter.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')